# Klasifikasi Kematangan Buah Pepaya Menggunakan SVM

Proyek ini bertujuan untuk mengklasifikasikan tingkat kematangan buah pepaya (mentah, setengah matang, matang) berdasarkan fitur warna menggunakan metode Support Vector Machine (SVM).

In [ ]:
# Import library yang diperlukan
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import os
from google.colab import files
import glob
from sklearn.model_selection import cross_val_score

## 1. Persiapan Dataset

Langkah pertama adalah menghubungkan ke Google Drive tempat dataset disimpan dan menentukan struktur folder dataset. Berdasarkan deskripsi tugas, dataset terdiri dari tiga kelas: mentah (16 buah), setengah matang (11 buah), dan matang (26 buah).

In [ ]:
# Mount Google Drive untuk mengakses dataset
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Tentukan path ke dataset
# Sesuaikan path ini dengan lokasi sebenarnya di Google Drive Anda
base_path = '/content/drive/MyDrive/papaya_dataset'  # Ganti dengan path sesuai struktur folder Anda
classes = ['mentah', 'setengah', 'matang']

print("Daftar kelas:", classes)
print("Jumlah data per kelas berdasarkan tugas:")
print("Mentah: 16, Setengah Matang: 11, Matang: 26")

## 2. Pra-pemrosesan Citra

Proses pra-pemrosesan penting untuk mempersiapkan citra agar siap untuk ekstraksi fitur. Langkah-langkah yang dilakukan sesuai dengan tugas:
- Normalisasi ukuran ke 224x224 piksel
- Reduksi noise dengan filter Gaussian
- Segmentasi untuk memisahkan buah dari latar belakang

In [ ]:
def preprocess_image(image_path):
    """Fungsi untuk melakukan pra-pemrosesan pada citra"""
    # Baca citra
    img = cv2.imread(image_path)
    
    # Resize citra ke ukuran 224x224
    img_resized = cv2.resize(img, (224, 224))
    
    # Terapkan filter Gaussian untuk mengurangi noise
    img_filtered = cv2.GaussianBlur(img_resized, (5, 5), 0)
    
    # Konversi ke ruang warna HSV untuk segmentasi
    hsv = cv2.cvtColor(img_filtered, cv2.COLOR_BGR2HSV)
    
    # Thresholding sederhana untuk memisahkan papaya dari latar belakang
    # Rentang HSV ini bisa disesuaikan tergantung warna dominan papaya
    lower_hsv = np.array([0, 20, 20])
    upper_hsv = np.array([180, 255, 255])
    mask = cv2.inRange(hsv, lower_hsv, upper_hsv)
    
    # Terapkan mask untuk mendapatkan citra yang disegmentasi
    segmented = cv2.bitwise_and(img_filtered, img_filtered, mask=mask)
    
    return segmented

## 3. Ekstraksi Fitur Warna

Ekstraksi fitur warna dilakukan menggunakan nilai rata-rata dari kanal RGB dan HSV. Ini sesuai dengan fokus proyek untuk membedakan tingkat kematangan berdasarkan warna (hijau untuk mentah, kuning/oranye untuk matang).

In [ ]:
def extract_features(image):
    """Fungsi untuk mengekstrak fitur warna dari citra"""
    # Hitung nilai rata-rata RGB
    mean_r = np.mean(image[:,:,2])  # R channel
    mean_g = np.mean(image[:,:,1])  # G channel
    mean_b = np.mean(image[:,:,0])  # B channel
    
    # Konversi ke HSV dan hitung nilai rata-rata
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mean_h = np.mean(hsv[:,:,0])  # Hue
    mean_s = np.mean(hsv[:,:,1])  # Saturation
    mean_v = np.mean(hsv[:,:,2])  # Value
    
    # Kembalikan fitur sebagai array
    features = [mean_r, mean_g, mean_b, mean_h, mean_s, mean_v]
    return features

## 4. Muat dan Proses Semua Citra

Proses ini mengiterasi semua citra dalam dataset, melakukan pra-pemrosesan, lalu mengekstrak fitur-fiturnya.

In [ ]:
# Muat dan proses semua citra
features_list = []
labels_list = []

# Proses untuk setiap kelas (mentah, setengah, matang)
for class_name in classes:
    class_path = os.path.join(base_path, class_name)
    print(f"Memproses kelas: {class_name}")
    
    # Cek apakah path kelas ada
    if os.path.exists(class_path):
        # Ambil semua file gambar dengan ekstensi tertentu
        image_extensions = ['*.jpg', '*.jpeg', '*.png']
        image_files = []
        
        for ext in image_extensions:
            image_files.extend(glob.glob(os.path.join(class_path, ext)))
        
        print(f"  Menemukan {len(image_files)} citra")
        
        # Proses setiap citra
        for img_path in image_files:
            try:
                processed_img = preprocess_image(img_path)
                features = extract_features(processed_img)
                features_list.append(features)
                labels_list.append(class_name)
            except Exception as e:
                print(f"  Gagal memproses {img_path}: {str(e)}")
                continue
    else:
        print(f"  Path tidak ditemukan: {class_path}")

# Konversi ke array numpy
X = np.array(features_list)
y = np.array(labels_list)

print(f"\nDataset berhasil diproses!")
print(f"Ukuran dataset: {X.shape}")
print(f"Jumlah sampel per kelas: {np.unique(y, return_counts=True)}")

## 5. Persiapan Data untuk Pelatihan

Encode label kelas dan pisahkan dataset menjadi data pelatihan dan pengujian.

In [ ]:
# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Pisahkan dataset menjadi data pelatihan dan pengujian (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

print(f"Ukuran data pelatihan: {X_train.shape}")
print(f"Ukuran data pengujian: {X_test.shape}")
print(f"Kelas: {label_encoder.classes_}")

## 6. Pelatihan Model SVM

Menggunakan Support Vector Machine (SVM) dengan kernel RBF untuk klasifikasi. SVM dipilih karena efektif untuk data dengan fitur numerik seperti fitur warna.

In [ ]:
# Buat dan latih model SVM
svm_model = svm.SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
svm_model.fit(X_train, y_train)

print("Model SVM selesai dilatih")

## 7. Evaluasi Model

Lakukan prediksi pada data pengujian dan hitung metrik evaluasi.

In [ ]:
# Lakukan prediksi
y_pred = svm_model.predict(X_test)

# Hitung akurasi
accuracy = accuracy_score(y_test, y_pred)
print(f"Akurasi model: {accuracy:.4f}")

# Tampilkan laporan klasifikasi
target_names = label_encoder.classes_
print("\nLaporan Klasifikasi:")
print(classification_report(y_test, y_pred, target_names=target_names))

## 8. Visualisasi Matriks Confusion

Visualisasi hasil klasifikasi dalam bentuk matriks confusion untuk memahami kinerja model secara lebih detail.

In [ ]:
# Tampilkan matriks confusion
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix - Klasifikasi Kematangan Pepaya')
plt.colorbar()
tick_marks = np.arange(len(target_names))
plt.xticks(tick_marks, target_names, rotation=45)
plt.yticks(tick_marks, target_names)

# Tambahkan anotasi teks ke matriks confusion
thresh = cm.max() / 2.
for i, j in np.ndindex(cm.shape):
    plt.text(j, i, format(cm[i, j], 'd'),
             horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")

plt.ylabel('Label Sebenarnya')
plt.xlabel('Label Prediksi')
plt.tight_layout()
plt.show()

## 9. Validasi Silang (Opsional)

Melakukan validasi silang untuk mendapatkan estimasi kinerja model yang lebih robust.

In [ ]:
# Lakukan validasi silang (5-fold)
cv_scores = cross_val_score(svm_model, X, y_encoded, cv=5)

print(f"Skor validasi silang (5-fold): {cv_scores}")
print(f"Rata-rata akurasi validasi silang: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

## 10. Ringkasan dan Kesimpulan

Menyimpulkan hasil yang diperoleh dari model klasifikasi.

In [ ]:
print("=== RINGKASAN HASIL KLASIFIKASI KEMATANGAN PAPAYA ===")
print(f"Jumlah total data: {len(X)}")
print(f"Jumlah fitur: {X.shape[1]} (RGB dan HSV mean)")
print(f"Jumlah kelas: {len(np.unique(y_encoded))}")
print(f"Akurasi model: {accuracy:.4f} ({accuracy*100:.2f}%)")

# Tampilkan distribusi kelas
unique, counts = np.unique(y, return_counts=True)
for i in range(len(unique)):
    print(f"Jumlah {unique[i]}: {counts[i]}")

print("\nModel SVM berhasil dilatih dan dievaluasi untuk mengklasifikasikan")
print("tingkat kematangan buah pepaya berdasarkan fitur warna.")